In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib

In [23]:
# 1. Memuat Dataset
data = pd.read_csv("Social_Network_Ads.csv")

In [24]:
# 2. Memeriksa Nilai yang Hilang
print("Nilai yang hilang per kolom:")
print(data.isnull().sum())

Nilai yang hilang per kolom:
User ID            0
Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64


In [25]:
# 3. Menangani Nilai yang Hilang
imputer = SimpleImputer(strategy='median')
data[['Age', 'EstimatedSalary']] = imputer.fit_transform(data[['Age', 'EstimatedSalary']])

In [26]:
# Cek apakah 'Gender' adalah kategorikal atau numerik
print("Unik nilai di kolom 'Gender':", data['Gender'].unique())


Unik nilai di kolom 'Gender': ['Male' 'Female']


In [27]:
# Jika 'Gender' adalah kategorikal, lakukan imputasi dengan modus
if data['Gender'].dtype == 'object':
    imputer = SimpleImputer(strategy='most_frequent')
    data['Gender'] = imputer.fit_transform(data[['Gender']]).ravel()

In [28]:
# 4. Encoding Fitur Kategorikal
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])

In [29]:
# 5. Normalisasi atau Skalasi Fitur Numerik
scaler = StandardScaler()
data[['Age', 'EstimatedSalary']] = scaler.fit_transform(data[['Age', 'EstimatedSalary']])

In [30]:
# 6. Menghapus Kolom yang Tidak Relevan (User ID)
data = data.drop(columns=['User ID'])  # Hapus kolom User ID

In [31]:
# 7. Menghapus Duplikasi
data = data.drop_duplicates()

In [32]:
# 8. Pembagian Data (Training dan Testing)
X = data.drop(columns=['Purchased'])  # Fitur, pastikan 'Purchased' adalah kolom target
y = data['Purchased']  # Target/label

# Split dataset menjadi 80% pelatihan dan 20% pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
# 9. Pemodelan Naive Bayes (GaussianNB)
nb = GaussianNB()

# Melatih model dengan data pelatihan
nb.fit(X_train, y_train)

# Melakukan prediksi dengan data pengujian
y_pred = nb.predict(X_test)

# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi: {accuracy * 100:.2f}%")

# Matriks kebingungannya
print("Matriks Kebingungannya:")
print(confusion_matrix(y_test, y_pred))

# Laporan Klasifikasi
print("Laporan Klasifikasi:")
print(classification_report(y_test, y_pred))

Akurasi: 92.11%
Matriks Kebingungannya:
[[42  6]
 [ 0 28]]
Laporan Klasifikasi:
              precision    recall  f1-score   support

           0       1.00      0.88      0.93        48
           1       0.82      1.00      0.90        28

    accuracy                           0.92        76
   macro avg       0.91      0.94      0.92        76
weighted avg       0.93      0.92      0.92        76



In [34]:
# 9.2. Hyperparameter Tuning dengan GridSearchCV (untuk GaussianNB)
param_grid = {'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]}
grid_search = GridSearchCV(GaussianNB(), param_grid, cv=5)

# Melatih model dengan data pelatihan
grid_search.fit(X_train, y_train)

# Melihat hasil terbaik
print("Hyperparameter terbaik:", grid_search.best_params_)
print(f"Akurasi terbaik: {grid_search.best_score_ * 100:.2f}%")

# Evaluasi model terbaik pada data uji
y_pred_best = grid_search.best_estimator_.predict(X_test)
print("Akurasi dengan hyperparameter terbaik:", accuracy_score(y_test, y_pred_best) * 100)

Hyperparameter terbaik: {'var_smoothing': 1e-09}
Akurasi terbaik: 87.81%
Akurasi dengan hyperparameter terbaik: 92.10526315789474


In [35]:
# 10. Menyimpan Model untuk Digunakan Kembali
joblib.dump(nb, 'naive_bayes_model.pkl')

# Untuk memuat model kembali nanti
model = joblib.load('naive_bayes_model.pkl')

In [36]:
# 11. Prediksi untuk Data Baru
# Misalnya, data baru yang ingin diprediksi
data_baru = pd.DataFrame({
    'Age': [30],
    'EstimatedSalary': [40000],
    'Gender': [0],  # Label encoding untuk 'Laki-laki'
})

# Melakukan preprocessing yang sama seperti pada data pelatihan
data_baru[['Age', 'EstimatedSalary']] = scaler.transform(data_baru[['Age', 'EstimatedSalary']])

# Pastikan hanya fitur yang digunakan dalam pelatihan yang diberikan untuk prediksi
# Mengurutkan kolom data_baru agar sesuai dengan urutan fitur pada X_train
data_baru = data_baru[X.columns]  # Menyesuaikan urutan fitur agar sesuai dengan X_train

# Melakukan prediksi dengan model
prediksi = nb.predict(data_baru)
print(f"Prediksi untuk data baru: {prediksi}")


Prediksi untuk data baru: [0]
